In [1]:
import ssl
import os
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
os.environ["REQUESTS_CA_BUNDLE"] = str(r"C:\Users\AL03238\OneDrive - Elevance Health\Documents\Work\Project Intelliauth\license\root.pem")
os.environ["SSL_CERT_FILE"] = str(r"C:\Users\AL03238\OneDrive - Elevance Health\Documents\Work\Project Intelliauth\license\root.pem")

In [2]:
!python --version

Python 3.10.10


## Model Download

In [3]:
!pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org chromadb 
# !pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose

In [4]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

In [5]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin"

In [6]:
# code for download the model from huggingface
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

In [7]:
# !pip install pymupdf

In [8]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyMuPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
import chromadb


In [9]:
# PINECONE_API_KEY = "7f6b7506-8283-42ff-8fce-c62007e0dfa1"
# PINE_CONE_HOST = "https://medical-chatbot-62dwz9e.svc.aped-4627-b74a.pinecone.io"

In [10]:
## Extract the data from pdf
def load_pdf(data):
    loader =DirectoryLoader(data, glob="*.pdf", loader_cls=PyMuPDFLoader)

    documents = loader.load()
    return documents

In [11]:
extracted_data = load_pdf('data/')

In [12]:
# extracted_data

In [13]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [14]:
text_chunks = text_split(extracted_data)
print(len(text_chunks))

5779


In [15]:
# download embedding model
def download_hugging_face_embedding():
    embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

# from sentence_transformers import SentenceTransformer

# def download_hugging_face_embedding():
#     model_name = "sentence-transformers/all-MiniLM-L6-v2"
#     embeddings = SentenceTransformer(model_name)
#     return embeddings


In [16]:
embeddings = download_hugging_face_embedding()

In [17]:
query_result = embeddings.embed_query("Hello World")
print("Length", len(query_result))

Length 384


ChromaDB

In [18]:
chroma_client = chromadb.Client()


In [19]:
collection = chroma_client.create_collection(name="medical_chatbot")


: 

In [20]:
collection.add(
    documents=[
        "This is a document about pineapple",
        "This is a document about oranges"
    ],
    ids=["id1", "id2"]
)


In [ ]:
results = collection.query(
    query_texts=["This is a query document about hawaii"], # Chroma will embed this for you
    n_results=2 # how many results to return
)
print(results)


## PineCone vectorDB initialisation

In [ ]:
PINECONE_API_KEY = "7f6b7506-8283-42ff-8fce-c62007e0dfa1"
PINE_CONE_HOST = "https://medical-chatbot-62dwz9e.svc.aped-4627-b74a.pinecone.io"

In [ ]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY, ssl_verify=False, proxy_url=None)

In [ ]:
index_name = "medical-chatbot"
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud='aws', region='us-east-1')
    )
index = pc.Index("medical-chatbot")


In [ ]:
index.describe_index_stats()


In [ ]:
def query(text):
    query_embedding = embeddings.embed_query(text)
    print(query_embedding)
    query_result = index.query(query_embedding, top_k=2, top_n=1)
    return query_result

In [ ]:
# from tqdm import tqdm
# for i, t in enumerate(tqdm(text_chunks)):
#     # Upsert each word embedding into the index
#     index.upsert(vectors=[{"id": str(i), "values": embeddings.embed_query(t.page_content), "metadata": {"text": t.page_content}}])

In [ ]:
prompt_template='''
Use the following pieces of information to answer the question:
If you don't know the answer to the question, just say you dont know, dont try to make up an answer to the question.

Context: {context}
Question: {question}
Only return the helpful answer to the question and nothing else.
Helpful answer: 
'''

In [ ]:
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"], output_variables=["helpful_answer"])
chain_type_kwargs = {"prompt":PROMPT}


In [ ]:
llm = CTransformers(model="model/llama-2-13b-chat.ggmlv3.q5_1.bin",
                    model_type = "llama",
                    config={"max_new_tokens":512,
                            'temperature':0.8})

In [ ]:
from langchain_pinecone import PineconeVectorStore

# Specify the name of the Pinecone index
# index_name = "langchain-test-index"

# Create a PineconeVectorStore from the index name
vectorstore = PineconeVectorStore(embedding=embeddings, index_name=index_name, pinecone_api_key=PINECONE_API_KEY)


In [ ]:
retriever  = vectorstore.as_retriever(search_type="similarity")

In [ ]:
retriever

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
# from langchain_openai import ChatOpenAI

# llm = ChatOpenAI()
# retriever = ...  # Your retriever

# Create the question-answer chain
question_answer_chain = create_stuff_documents_chain(llm, prompt=PROMPT)
retrieval_qa_chain = create_retrieval_chain(retriever, question_answer_chain)

# Run the retrieval QA chain with the query
# result = retrieval_qa_chain
result = retrieval_qa_chain.invoke({"input": "What are allergies?","context": "you are a medical doctor", "question": "what are allergies?"})

# Access the answer from the result
answer = result["answer"]


In [ ]:
print(answer)

In [ ]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(  
    llm=llm,  
    chain_type="stuff",  
    retriever=retriever
)  
qa.run("What are allergies?")

In [ ]:
from langchain.chains.retrieval_qa.base import RetrievalQA

In [ ]:

class MyRetrievalQA(RetrievalQA):
    def _get_relevant_documents(self, query):
        # Implement your logic to retrieve relevant documents based on the query
        # Return the relevant documents as a list
        relevant_documents = [...]
        return relevant_documents

In [ ]:
from langchain.schema.retriever import BaseRetriever, Document
from typing import List

class CustomRetriever(BaseRetriever):
    def _get_relevant_documents(
        self, query: str
    ) -> List[Document]:
        # Use your existing retriever to get the documents
        documents = self.retriever._get_relevant_documents(query, callbacks=run_manager.get_child())
        
        # Sort the documents by "source"
        documents = sorted(documents, key=lambda doc: doc.metadata.get('source'))
        
        return documents

In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm, 
                                 chain_type="stuff",
                                 retriever=CustomRetriever(vectorstore.as_retriever(search_kwargs={"k":2})), 
                                 chain_type_kwargs=chain_type_kwargs)

In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, chain_type="stuff", retriever=vectorstore.as_retriever(), chain_type_kwargs=chain_type_kwargs,return_source_documents=True 
)

In [ ]:
print(vectorstore)

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=vectorstore.as_retriever(vectorstore, t), chain_type_kwargs=chain_type_kwargs
)